# Cross-subject decoding Motor responses 
### (LINDEX v. LMID and RINDEX v. RMID)

#### Takuya Ito
#### 2/28/2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa_python3 as tools_group
from importlib import reload
import nibabel as nib

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


## 0.1 Load data

In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(list(xticks.keys()))
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

In [3]:
# gsr = True
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools_group.loadMotorResponses(subj,hand='Left')
    scount += 1

## 1.1 Run across subject decoding on hand-specific motor responses

In [4]:
nproc = 30
# ncvs = 50
tools_group = reload(tools_group)
ncvs = 1
rois = np.where(networkdef==networkmappings['smn'])[0]
print('Running decoding for Right hand responses')
distances_baseline_rh, rmatches_rh, rmismatches_rh = tools_group.conditionDecodings(data_task_rh, rois, ncvs=ncvs, effects=True, motorOutput=True, nproc=nproc)
print('Running decoding for Left hand responses')
distances_baseline_lh, rmatches_lh, rmismatches_lh = tools_group.conditionDecodings(data_task_lh, rois, ncvs=ncvs, effects=True, motorOutput=True, nproc=nproc)



Running decoding for Right hand responses


/mnt/linuxlocal2/projects3/SRActFlow/docs/scripts/tools_group_rsa_python3.py:295: RuntimeWarning: Mean of empty slice
  trainset_downsampled[:,1] = np.nanmean(trainset[:,feat2_mask],axis=1)
/mnt/linuxlocal2/projects3/SRActFlow/docs/scripts/tools_group_rsa_python3.py:300: RuntimeWarning: Mean of empty slice
  testset_downsampled[:,1] = np.nanmean(testset[:,feat2_mask],axis=1)
/mnt/linuxlocal2/projects3/SRActFlow/docs/scripts/tools_group_rsa_python3.py:294: RuntimeWarning: Mean of empty slice
  trainset_downsampled[:,0] = np.nanmean(trainset[:,feat1_mask],axis=1)
/mnt/linuxlocal2/projects3/SRActFlow/docs/scripts/tools_group_rsa_python3.py:299: RuntimeWarning: Mean of empty slice
  testset_downsampled[:,0] = np.nanmean(testset[:,feat1_mask],axis=1)


Running decoding for Left hand responses


## 1.2 Compute statistics

####  Right-hand decoding statistics

In [5]:
smnROIs = np.where(networkdef==networkmappings['smn'])[0] 

ntrials = distances_baseline_rh.shape[1] 
statistics_rh = np.zeros((len(smnROIs),3)) # acc, q, acc_thresh
for roicount in range(len(smnROIs)):
    p = stats.binom_test(np.mean(distances_baseline_rh[roicount,:])*ntrials,n=ntrials,p=1/float(data_task_rh.shape[1]))
    if np.mean(distances_baseline_rh[roicount,:])>1/float(data_task_rh.shape[1]):
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_rh[roicount,0] = np.mean(distances_baseline_rh[roicount,:])
    statistics_rh[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_rh[:,1])
for roicount in range(len(smnROIs)):
    statistics_rh[roicount,1] = qs[roicount]
    statistics_rh[roicount,2] = h0[roicount]*statistics_rh[roicount,0]

    

In [11]:
rmatches_rh[sig_ind]

TypeError: only integer scalar arrays can be converted to a scalar index

In [14]:
# Count number of significant ROIs for RH decoding
sig_ind = np.where(statistics_rh[:,1]<0.05)[0]
sig_ind = np.asarray(sig_ind,dtype=int)
print('Number of ROIs significant for right hand responses:', sig_ind.shape[0])
if sig_ind.shape[0]>0:
    print('Significant ROIs:', smnROIs[sig_ind]+1)
#     print('R_matched effect-size:', rmatches_rh[sig_ind])
#     print('R_mismatched effect-size:', rmismatches_rh[sig_ind])
print('Accuracies:', statistics_rh[sig_ind,0])

Number of ROIs significant for right hand responses: 15
Significant ROIs: [  8   9  51  52  53  54  56 101 188 189 220 221 236 281 295]
Accuracies: [0.77083333 0.83333333 0.66666667 0.625      0.80208333 0.67708333
 0.65625    0.58333333 0.625      0.63541667 0.58333333 0.64583333
 0.66666667 0.58333333 0.59375   ]


####  Left-hand decoding statistics

In [15]:
smnROIs = np.where(networkdef==networkmappings['smn'])[0] 

ntrials = distances_baseline_lh.shape[1] 
statistics_lh = np.zeros((len(smnROIs),3)) # acc, q, acc_thresh
for roicount in range(len(smnROIs)):
    p = stats.binom_test(np.mean(distances_baseline_lh[roicount,:])*ntrials,n=ntrials,p=1/float(data_task_lh.shape[1]))
    if np.mean(distances_baseline_lh[roicount,:])>1/float(data_task_lh.shape[1]):
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_lh[roicount,0] = np.mean(distances_baseline_lh[roicount,:])
    statistics_lh[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_lh[:,1])
for roicount in range(len(smnROIs)):
    statistics_lh[roicount,1] = qs[roicount]
    statistics_lh[roicount,2] = h0[roicount]*statistics_lh[roicount,0]

    

In [16]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_lh[:,1]<0.05)[0]
print('Number of ROIs significant for right hand responses:', sig_ind.shape[0])
if sig_ind.shape[0]>0:
    print('Significant ROIs:', smnROIs[sig_ind] + 1)
#     print 'R_matched effect-size:', rmatches_lh[sig_ind]
#     print 'R_mismatched effect-size:', rmismatches_lh[sig_ind]
print('Accuracies:', statistics_lh[sig_ind,0])

Number of ROIs significant for right hand responses: 11
Significant ROIs: [ 41  51  54 102 188 189 220 231 232 233 284]
Accuracies: [0.59375    0.59375    0.61458333 0.65625    0.73958333 0.89583333
 0.59375    0.76041667 0.625      0.70833333 0.625     ]


## 1.3 Map accuracies back to cortical surface

In [27]:
# Put all data into a single matrix (since we only run a single classification)
lefthand = np.zeros((glasser2.shape[0],3))
righthand = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in smnROIs:
    # Print significant parcel number
    vertex_ind = np.where(glasser2==roi+1)[0]
    lefthand[vertex_ind,0] = statistics_lh[roicount,0]
    lefthand[vertex_ind,1] = statistics_lh[roicount,1]
    lefthand[vertex_ind,2] = statistics_lh[roicount,2]

    righthand[vertex_ind,0] = statistics_rh[roicount,0]
    righthand[vertex_ind,1] = statistics_rh[roicount,1]
    righthand[vertex_ind,2] = statistics_rh[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/MotorResponseRegions_LH.csv', smnROIs[np.where(statistics_lh[:,1]<0.05)[0]], delimiter=',')
np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/MotorResponseRegions_RH.csv', smnROIs[np.where(statistics_rh[:,1]<0.05)[0]], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
filename = 'MotorResponseDecodingsLH'
np.savetxt(outdir + filename + '.csv', lefthand,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)

filename = 'MotorResponseDecodingsRH'
np.savetxt(outdir + filename + '.csv', righthand,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0